# Object Detection
This notebook will serve as the primary means of documentation for creating an object detection model
_______________________________________________________________________________________________________________________________________________

### Full group includes:
- Jordan Brown
- Dylan Roy
- Maxwell Barret
- Julia Dewhurt

### Primary notebook contributers:
- ""
- ""
_______________________________________________________________________________________________________________________________________________

***The primary notebook contributers are the group members who were assigned to this specific task. All group members will work in collaboration to create a final working product. However, the nature of this project calls for the full group to be assigned to primary tasks.***

## Notebook Goal/Purpose

This notebook will be used to create a model for traffic based object detection. This will involve using labeled image and video feeds to determine what traffic objects look like. This model will be able to detect and track traffic counts at individual intersections. This tracking will be used to create a dataset which can hopefully be used to train a traffic prediction for the traffic analysis portion of this project.
_______________________________________________________________________________________________________________________________________________

Code cells should use the following layout template
```
"""
At the top will be a commment block
this comment block will describe the
primary purpose for the cells code.
"""

# A comment stating used libary(s)

Code
    Implementation
        Goes
            Here

```

In [ ]:
"""
This cell will contain the import statements for the notebook.
"""
from kaggle.api.kaggle_api_extended import KaggleApi
import os
import json
import pandas as pd

In [ ]:
# from kaggle.api.kaggle_api_extended import KaggleApi
# import os

# Initialize the API
api = KaggleApi()
api.authenticate()

# Dataset details
dataset = "imtkaggleteam/city-intersection-computer-vision"
download_path = "dataset"  # Desired directory

# Ensure the download path exists
if not os.path.exists(download_path):
    os.makedirs(download_path, exist_ok=True)

    # Download the dataset
    api.dataset_download_files(dataset, path=download_path, unzip=True)

    print(f"Dataset downloaded to: {os.path.abspath(download_path)}")

In [ ]:
import sys, os, distutils.core
from IPython.display import clear_output

!python -m pip install pyyaml==5.1
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))
clear_output()

In [ ]:
# Importing dependencies

import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
print("detectron2 version:", detectron2.__version__)

import numpy as np
import pandas as pd
import os, json, cv2, random
from IPython import display
import PIL
import matplotlib.pyplot as plt

# importing detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


SEED = 12
THRESHOLD = 0.9

In [ ]:
# a function that used for creation a coco json file from a csv file
# original reference -> https://stackoverflow.com/questions/62545034/how-to-convert-a-csv-table-into-coco-format-in-python

def create_csv_to_coco(path, save_json_path):
    data = pd.read_csv(path)
    data = data[['filename','class','width', 'height','xmin','ymin','xmax','ymax']]

    images = []
    categories = []
    annotations = []

    category = {}
    category["supercategory"] = 'none'
    category["id"] = 0
    category["name"] = 'None'
    categories.append(category)

    data['fileid'] = data['filename'].astype('category').cat.codes
    data['categoryid']= pd.Categorical(data['class'],ordered= True).codes
    data['categoryid'] = data['categoryid']+1
    data['annid'] = data.index
    
    def image(row):
        image = {}
        image["height"] = row.height
        image["width"] = row.width
        image["id"] = row.fileid
        image["file_name"] = row.filename
        return image

    def category(row):
        category = {}
        category["supercategory"] = 'None'
        category["id"] = row.categoryid
        category["name"] = row[2]
        return category

    def annotation(row):
        annotation = {}
        area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
        annotation["segmentation"] = []
        annotation["iscrowd"] = 0
        annotation["area"] = area
        annotation["image_id"] = row.fileid

        annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]

        annotation["category_id"] = row.categoryid
        annotation["id"] = row.annid
        return annotation

    for row in data.itertuples():
        annotations.append(annotation(row))

    imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
    for row in imagedf.itertuples():
        images.append(image(row))

    catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
    for row in catdf.itertuples():
        categories.append(category(row))

    data_coco = {}
    data_coco["images"] = images
    data_coco["categories"] = categories
    data_coco["annotations"] = annotations
    json.dump(data_coco, open(save_json_path, "w"), indent=4)
    print( save_json_path ,' file created...')

In [ ]:
create_csv_to_coco(f'{download_path}/train/_annotations.csv', 
                   'dataset/train_coco.json')
create_csv_to_coco(f'{download_path}/valid/_annotations.csv', 
                   'dataset/valid_coco.json')
create_csv_to_coco(f'{download_path}/test/_annotations.csv', 
                   'dataset/test_coco.json')


In [ ]:
# Creating coco instances

from detectron2.data.datasets import register_coco_instances
from detectron2.structures import BoxMode

    
register_coco_instances(f"city_intersection_train_dataset", {},
                        f"../working/train_coco.json",
                        f"../input/city-intersection-computer-vision/train")

register_coco_instances(f"city_intersection_valid_dataset", {},
                        f"../working/valid_coco.json",
                        f"../input/city-intersection-computer-vision/valid")

register_coco_instances(f"city_intersection_test_dataset", {},
                        f"../working/test_coco.json",
                        f"../input/city-intersection-computer-vision/test")


In [ ]:
# Examples from the training dataset

import random
from detectron2.utils.visualizer import Visualizer

my_dataset_train_metadata = MetadataCatalog.get("city_intersection_train_dataset")
train_dataset_dicts = DatasetCatalog.get("city_intersection_train_dataset")

# A function that creates examples from the dataset
def create_random_images(dataset_dict,dataset_metadata, seed, image_scale = 0.7):
    np.random.seed(seed)
    images = np.random.permutation(dataset_dict)[:2]
    
    fig, axs = plt.subplots(1,2, figsize = (12,6), dpi = 100)
    for i in range(2):
        im = images[i]
        img_link = im['file_name']
        img_id = im['image_id']
        img = cv2.imread(img_link)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        visualizer = Visualizer(img, metadata= dataset_metadata, scale=image_scale)
        vis = visualizer.draw_dataset_dict(im)
        final_img = vis.get_image()
        
        axs[i].set_title('image id: ' + str(img_id), fontsize = 10)
        axs[i].axis('off')
        axs[i].imshow(final_img)

In [ ]:
# Examples from the training dataset

create_random_images(train_dataset_dicts, my_dataset_train_metadata, seed = 92, image_scale = 1)
create_random_images(train_dataset_dicts, my_dataset_train_metadata, seed = 1, image_scale = 1)